## ハイパーパラメータチューニング

ハイパーパラメータチューニングは学習を何度も繰り返す必要があり、非常に時間がかかる作業になります。  
仮に1回の学習ループが3日かかるとしたら、パラメータを数回変えて試してみるだけで非常に時間がかかってしまいます。

今回はその作業をお金の力で解決してしまう方法を学習します。

## コードのモジュール化

まず、コードを何度も実行するためにはjupyter上で処理するのはあまり向かないため、scriptに落とし込みます。  
コードのコアの部分を移動したコードが`mfashion_keras/model.py`にあります。  
そして、jobのkickerとなるコードを`mfashion_keras/task.py`に記載してあります。  
試しに実行してみましょう

In [1]:
# このハンズオンで必要なライブラリのインストール
!pip install cloudml-hypertune

In [2]:
!python3 -m mfashion_keras.task --output_dir=./output --model=cnn --batch_size=64 --batch_norm

output_dir: ./output/
2022-04-27 11:10:13.820928: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:936] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-04-27 11:10:13.838518: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:936] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-04-27 11:10:13.839209: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:936] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-04-27 11:10:13.840225: I tensorflow/core/platform/cpu_feature_guard.cc:151] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appr

## gcloudコマンドを用いたVertex AIでのtraining実行

下記のコマンドをgcloudのVertex AI経由で実行することで、Vertex AI内のリソースを使ってtrainingを実行することができます。

In [3]:
## 書き換える
USER = "<<username>>"

## 必要に応じて書き換える
MODEL_TYPE = "cnn"
LEARNING_RATE = 0.01
BATCH_SIZE = 64
TRAIN_STEPS = 1000

## 書き換えなくて良い (出力先を変えたい場合はOUTPUT_DIRを変更)
BUCKET = "mixi-ml-handson-2022"
REGION = "asia-northeast1"
OUTPUT_DIR = "gs://" + BUCKET + "/" + USER + "/mfashion/trained_" + MODEL_TYPE

In [ ]:
%%bash -s "$USER" "$BUCKET" "$REGION" "$MODEL_TYPE" "$LEARNING_RATE" "$BATCH_SIZE" "$TRAIN_STEPS" "$OUTPUT_DIR"

DATE=`date +%Y%m%d_%H%M%S`
DISPLAY_NAME=mfashion_$4_$1_$DATE

echo $8
echo ${DISPLAY_NAME}

gcloud ai custom-jobs create \
  --region=$3 \
  --display-name=${DISPLAY_NAME} \
  --args="--output_dir",$8,"--train_steps",$7,"--model",$4,\
"--learning_rate",$5,"--batch_size",$6 \
  --worker-pool-spec=machine-type=n1-standard-4,replica-count=1,accelerator-type=NVIDIA_TESLA_T4,\
executor-image-uri=asia-docker.pkg.dev/vertex-ai/training/tf-gpu.2-8:latest,local-package-path=./mfashion_keras,python-module=task


上記のコードの実行には数分の時間がかかります。

実行が確認できたら、gcpのコンソールから`vertex AI >> トレーニング >> CUSTOM JOBS`に遷移した後、  
リージョンを`asia-northeast1`にして作ったjobが表示されているか確認してみてください。  
(実行ログは該当のjobに遷移した後、`ログを表示`ボタンで確認できます。)

これでjobを作成し、自分のマシン以外のリソースを使って実行できました。  
上記のコマンドはjupyter上で実行する必要もないため、もちろんコマンドラインから実行しても同様に実行が可能です。
  
そしてパラメータを変えながら大量にjobを並列実行すれば、最適なパラメータチューニングをすることが可能になります。

## Vertex AIでのハイパーパラメータチューニング
　
各クラウドで似たような仕組みはありますが、今回はgcloudのパラメータチューニングを使用してチューニングします。

In [5]:
## 必要に応じて書き換える
MODEL_TYPE = "cnn"
LEARNING_RATE = 0.01
BATCH_SIZE = 64
TRAIN_STEPS = 1000

In [6]:
from IPython.core.magic import register_line_cell_magic

## '%%writefile'にglobal変数を読みこませるカスタムマジックコマンド'%%writetemplate'を定義
@register_line_cell_magic
def writetemplate(line, cell):
    with open(line, 'w') as f:
        f.write(cell.format(**globals()))   

最適値を探すのに、`Manual`, `Grid Search`, `Random Search`, `Baysean Search`の4つの探索方法が用意されています。  
[詳しくはここ](https://cloud.google.com/vertex-ai/docs/reference/rest/v1/projects.locations.hyperparameterTuningJobs)や[ここ](https://cloud.google.com/vertex-ai/docs/reference/rest/v1/StudySpec)を確認してください。

以下はGrid Searchを用いた例です。 
今回はlearning_rateを最初値0.001から最大値0.3までの間を探索してみます。

In [7]:
%%writetemplate hyperparam.yaml
studySpec:
  metrics:
  - metricId: ccentropy
    goal: MINIMIZE
  parameters:
  - parameterId: learning_rate
    discreteValueSpec:
      values: [0.001, 0.005, 0.01, 0.05, 0.1, 0.3]
  algorithm: GRID_SEARCH
  decayCurveStoppingSpec:
    useElapsedDuration: True
trialJobSpec:
  workerPoolSpecs:
  - machineSpec:
      machineType: n1-standard-4
    replicaCount: 1
    pythonPackageSpec:
      executorImageUri: asia-docker.pkg.dev/vertex-ai/training/tf-cpu.2-8:latest
      packageUris: [gs://{BUCKET}/{USER}/mfashion/src/mfashion_keras-1.0.tar.gz]
      pythonModule: mfashion_keras.task
      args: [--output_dir, {OUTPUT_DIR}, --train_steps, "{TRAIN_STEPS}", --model, {MODEL_TYPE}, --batch_size, "{BATCH_SIZE}"]

In [8]:
%%bash -s "$USER" "$BUCKET"

python3 setup.py sdist --formats=gztar 
gsutil cp dist/mfashion_keras-1.0.tar.gz gs://$2/$1/mfashion/src/

running sdist
running egg_info
creating mfashion_keras.egg-info
writing mfashion_keras.egg-info/PKG-INFO
writing dependency_links to mfashion_keras.egg-info/dependency_links.txt
writing requirements to mfashion_keras.egg-info/requires.txt
writing top-level names to mfashion_keras.egg-info/top_level.txt
writing manifest file 'mfashion_keras.egg-info/SOURCES.txt'
reading manifest file 'mfashion_keras.egg-info/SOURCES.txt'
writing manifest file 'mfashion_keras.egg-info/SOURCES.txt'
running check
creating mfashion_keras-1.0
creating mfashion_keras-1.0/mfashion_estimator
creating mfashion_keras-1.0/mfashion_keras
creating mfashion_keras-1.0/mfashion_keras.egg-info
copying files to mfashion_keras-1.0...
copying setup.py -> mfashion_keras-1.0
copying mfashion_estimator/__init__.py -> mfashion_keras-1.0/mfashion_estimator
copying mfashion_estimator/model.py -> mfashion_keras-1.0/mfashion_estimator
copying mfashion_estimator/task.py -> mfashion_keras-1.0/mfashion_estimator
copying mfashion_kera




Copying file://dist/mfashion_keras-1.0.tar.gz [Content-Type=application/x-tar]...
/ [1 files][  3.1 KiB/  3.1 KiB]                                                
Operation completed over 1 objects/3.1 KiB.                                      


In [11]:
%%bash -s "$USER" "$REGION" "$MODEL_TYPE"

DATE=`date +%Y%m%d_%H%M%S`
DISPLAY_NAME=mfashion_$3_$1_$DATE

gcloud ai hp-tuning-jobs create \
  --region=$2 \
  --config=hyperparam.yaml \
  --display-name=${DISPLAY_NAME} \
  --max-trial-count=6 \
  --parallel-trial-count=6

Using endpoint [https://asia-northeast1-aiplatform.googleapis.com/]
Hyperparameter tuning job [949634998770597888] submitted successfully.

Your job is still active. You may view the status of your job with the command

  $ gcloud ai hp-tuning-jobs describe 949634998770597888 --region=asia-northeast1

Job State: JOB_STATE_PENDING


実行が確認できたら、先ほどと同じようにgcpのコンソールから`vertex AI >> トレーニング >> HYPERPARAMETER TUNING JOBS`に遷移した後、  
リージョンを`asia-northeast1`にして作ったjobが表示されているか確認してみてください。

また、今回は`ccentropy`という指標を評価に使いましたが、これを変更するにはどうすればいいのでしょうか？確認してみてください。